In [6]:
import abc
import numpy as np
import jax.numpy as jnp
from jax import jacfwd, hessian, jit
from manifpy import SE3, SE3Tangent, SO3, SO3Tangent

In [14]:
def skew( w ):
    if isinstance(w, np.ndarray) or isinstance(w, jnp.ndarray) and w.shape == (3,) or w.shape == (3, 1):
        return np.array([
            [0, -w[2], w[1]],
            [w[2], 0, -w[0]],
            [-w[1], w[0], 0]
        ])
    else:
        raise ValueError("Input must be a 3d vector")

w = np.array([1., 2., 3.])
print(skew(w))

[[ 0. -3.  2.]
 [ 3.  0. -1.]
 [-2.  1.  0.]]


In [15]:
def skew( w ):
    return SO3Tangent(w).hat()

w = np.array([1, 2, 3])
print(skew(w))

[[ 0. -3.  2.]
 [ 3.  0. -1.]
 [-2.  1.  0.]]


In [16]:
def adjoint(x):
    w = np.array([x[0], x[1], x[2]])
    v = np.array([x[3], x[4], x[5]])
    adx = np.block([
        [skew(w), np.zeros((3, 3))],
        [skew(v), skew(w)]
    ])
    return adx

x = [1, 2, 3, 4, 5, 6]
result = adjoint(x)
print(result)

[[ 0. -3.  2.  0.  0.  0.]
 [ 3.  0. -1.  0.  0.  0.]
 [-2.  1.  0.  0.  0.  0.]
 [ 0. -6.  5.  0. -3.  2.]
 [ 6.  0. -4.  3.  0. -1.]
 [-5.  4.  0. -2.  1.  0.]]


In [17]:
class BaseDynamics():

    """Dynamics Model."""

    @property
    @abc.abstractmethod
    def state_size(self):
        """State size."""
        raise NotImplementedError

    @property
    @abc.abstractmethod
    def action_size(self):
        """Action size."""
        raise NotImplementedError

    @property
    @abc.abstractmethod
    def has_hessians(self):
        """Whether the second order derivatives are available."""
        raise NotImplementedError

    @abc.abstractmethod
    def f(self, x, u, i):
        """Dynamics model.

        Args:
            x: Current state [state_size].
            u: Current control [action_size].
            i: Current time step.

        Returns:
            Next state [state_size].
        """
        raise NotImplementedError

    @abc.abstractmethod
    def f_x(self, x, u, i):
        """Partial derivative of dynamics model with respect to x.

        Args:
            x: Current state [state_size].
            u: Current control [action_size].
            i: Current time step.

        Returns:
            df/dx [state_size, state_size].
        """
        raise NotImplementedError

    @abc.abstractmethod
    def f_u(self, x, u, i):
        """Partial derivative of dynamics model with respect to u.

        Args:
            x: Current state [state_size].
            u: Current control [action_size].
            i: Current time step.

        Returns:
            df/du [state_size, action_size].
        """
        raise NotImplementedError

    @abc.abstractmethod
    def f_xx(self, x, u, i):
        """Second partial derivative of dynamics model with respect to x.

        Note:
            This is not necessary to implement if you're planning on skipping
            Hessian evaluation as the iLQR implementation does by default.

        Args:
            x: Current state [state_size].
            u: Current control [action_size].
            i: Current time step.

        Returns:
            d^2f/dx^2 [state_size, state_size, state_size].
        """
        raise NotImplementedError

    @abc.abstractmethod
    def f_ux(self, x, u, i):
        """Second partial derivative of dynamics model with respect to u and x.

        Note:
            This is not necessary to implement if you're planning on skipping
            Hessian evaluation as the iLQR implementation does by default.

        Args:
            x: Current state [state_size].
            u: Current control [action_size].
            i: Current time step.

        Returns:
            d^2f/dudx [state_size, action_size, state_size].
        """
        raise NotImplementedError

    @abc.abstractmethod
    def f_uu(self, x, u, i):
        """Second partial derivative of dynamics model with respect to u.

        Note:
            This is not necessary to implement if you're planning on skipping
            Hessian evaluation as the iLQR implementation does by default.

        Args:
            x: Current state [state_size].
            u: Current control [action_size].
            i: Current time step.

        Returns:
            d^2f/du^2 [state_size, action_size, action_size].
        """
        raise NotImplementedError
    

In [ ]:
class ErrorStateSE3Dynamics(BaseDynamics):

    """Error-State SE(3) Dynamics Model implemented with Jax."""

    def __init__(self, X_ref, xi_ref, state_size=13, action_size=6, hessians=False, **kwargs):
        """Constructs an Dynamics model for SE(3).

        Args:
            f: Discretized dynamics function for velocity (e.g. after RK4):
                Args:
                    x: Batch of state variables, stacked by
                        Lie algebra, perturbed Lie algebra  
                    u: Batch of action variables, the generalized control 
                        inputs in cotangent space of Lie algebra
                    i: Batch of time step variables.
                Returns:
                    f: Batch of next state variables, stacked by
                        Lie algebra, perturbed Lie algebra  
            state_size: State variable dimension.
            action_size: Input variable dimension.
            hessians: Evaluate the dynamic model's second order derivatives.
                Default: only use first order derivatives. (i.e. iLQR instead
                of DDP).
            **kwargs: Additional keyword-arguments to pass to any potential 
                function, e.g. in the preivious version `theano.function()`.
        """

        self._state_size = state_size
        self._action_size = action_size

        self._has_hessians = hessians
        # if hessians:
        #     self._f_xx = jit(hessian(f, argnums=0))
        #     self._f_ux = jit(jacfwd( jacfwd(f, argnums=1) ))
        #     self._f_uu = jit(hessian(f, argnums=1))

        super(SE3Dynamics, self).__init__()

    @property
    def state_size(self):
        """State size."""
        return self._state_size

    @property
    def action_size(self):
        """Action size."""
        return self._action_size

    @property
    def has_hessians(self):
        """Whether the second order derivatives are available."""
        return self._has_hessians
    
    def skew( self, w ):
        """Get the isomorphic element in the Lie algebra for SO3, 
            i.e. the skew symmetric matrix"""
        if isinstance(w, np.ndarray) or isinstance(w, jnp.ndarray) and w.shape == (3,) or w.shape == (3, 1):
            return np.array([
                [0, -w[2], w[1]],
                [w[2], 0, -w[0]],
                [-w[1], w[0], 0]
            ])
        else:
            raise ValueError("Input must be a 3d vector")

    def adjoint( self, xi ):
        """ Get the the adjoint matrix representation of Lie Algebra"""
        w = np.array([xi[0], xi[1], xi[2]])
        v = np.array([xi[3], xi[4], xi[5]])
        adx = np.block([
            [self.skew(w), np.zeros((3, 3))],
            [self.skew(v), self.skew(w)]
        ])
        return adx
    
    def adjoint( self, xi ):
        """ Get the the adjoint matrix representation of Lie Algebra"""
        return self.adjoint(xi).T

    def f(self, x, u, i):
        """Dynamics model.

        Args:
            x: Current state [state_size], stacked by
                Lie algebra, perturbed Lie algebra  
            u: Current control [action_size].
            i: Current time step.

        Returns:
            Next state [state_size].
        """

        

        return self._f(x,u,i)

    def f_x(self, x, u, i):
        """Partial derivative of dynamics model with respect to x.

        Args:
            x: Current state [state_size].
            u: Current control [action_size].
            i: Current time step.

        Returns:
            df/dx [state_size, state_size].
        """
        return self._f_x(x,u,i)

    def f_u(self, x, u, i):
        """Partial derivative of dynamics model with respect to u.

        Args:
            x: Current state [state_size].
            u: Current control [action_size].
            i: Current time step.

        Returns:
            df/du [state_size, action_size].
        """
        return self._f_u(x,u,i)

    def f_xx(self, x, u, i):
        """Second partial derivative of dynamics model with respect to x.

        Args:
            x: Current state [state_size].
            u: Current control [action_size].
            i: Current time step.

        Returns:
            d^2f/dx^2 [state_size, state_size, state_size].
        """
        if not self._has_hessians:
            raise NotImplementedError

        return self._f_xx(x,u,i)

    def f_ux(self, x, u, i):
        """Second partial derivative of dynamics model with respect to u and x.

        Args:
            x: Current state [state_size].
            u: Current control [action_size].
            i: Current time step.

        Returns:
            d^2f/dudx [state_size, action_size, state_size].
        """
        if not self._has_hessians:
            raise NotImplementedError

        return self._f_ux(x,u,i)

    def f_uu(self, x, u, i):
        """Second partial derivative of dynamics model with respect to u.

        Args:
            x: Current state [state_size].
            u: Current control [action_size].
            i: Current time step.

        Returns:
            d^2f/du^2 [state_size, action_size, action_size].
        """
        if not self._has_hessians:
            raise NotImplementedError

        return self._f_uu(x,u,i)

In [ ]:
def f(x, u, i):
    """Dynamics model.

    Args:
        x: Current state [state_size], stacked by
            Lie algebra, perturbed Lie algebra  
        u: Current control [action_size].
        i: Current time step.

    Returns:
        Next state [state_size].
    """

    A = np.block[ -self.adjoint( xid ), np.identity(  ) ]

    return 

In [12]:
np.identity(3)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])